# Initial commit

Explanation on generator design choices. Don't look if you don't want spoilers!

### Popular times (by hour)

Used Google popular times to estimate orders per hour. Assigned weights and assumed 0 value gives 50%.

Huge variance - busiest day is almost 3x as busy as least busy day.

### Market sizing

Seating count, dining time, delivery/takeout

### Annual/seasonal multipliers

Year multiplier, summer/winter month multiplier. Mostly negative.

### Chance for additional item in order

If ordering starter or dessert, there's a chance for an additional item. 

### Chance for increased order quantity

### Generating by hour

Tried by month (because csvs are by month), but "accuracy" to the hour made more sense. Added variance with normal distribution.

### Weights for menu (by category, by item)

Categories have weights, and then within the category the items have weights.